In [2]:
library(tidyverse)

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ readr     2.1.5
✔ forcats   1.0.0     ✔ stringr   1.5.1
✔ ggplot2   3.5.1     ✔ tibble    3.2.1
✔ lubridate 1.9.3     ✔ tidyr     1.3.1
✔ purrr     1.0.2     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors


# DSCI Project Individual Portion
A real data science project from a UBC research group is researching how people play video games through recording players' actions when playing on a MineCraft server. However, due to limited resources, the study requires a narrower recruitment team of players that would provide them a greater amount of data. Hence, this page investigate in the narrower scope; more specifically, investigate traits players obtain that are most likely to contribute a large amount of data.

The data "players.csv" and "sessions.csv" that this project is working on is stored in data file. Individually, they have the following variables:


**1. "players.csv" is dataset with 7 variables, each describing each players (one row per unique player) with higher-level attributes.**
|Variable Name |Type|Meaning|
|:------------:|:--:|:-----:|
|`experience`  |chr |Experience with MineCraft (beginner, regular, amateur, pro, veteran ordered in increasing experience)|
|`subscribe`   |lgl |Subscribe to the Newspaper or not (True/False)|
|`hashedEmail` |chr |Registration email|
|`played_hours`|dbl |Hours played on server (hours)|
|`name`        |chr |Player's name|
|`gender`      |chr |Player's gender|
|`age`         |dbl |Player's age|

### Overview of "player_data.csv" Table

In [3]:
player_data <- read_csv("data/players.csv")
head(player_data)

Rows: 196 Columns: 7
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (4): experience, hashedEmail, name, gender
dbl (2): played_hours, Age
lgl (1): subscribe

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


experience,subscribe,hashedEmail,played_hours,name,gender,Age
<chr>,<lgl>,<chr>,<dbl>,<chr>,<chr>,<dbl>
Pro,TRUE,f6daba428a5e19a3d47574858c13550499be23603422e6a0ee9728f8b53e192d,30.3,Morgan,Male,9
Veteran,TRUE,f3c813577c458ba0dfef80996f8f32c93b6e8af1fa939732842f2312358a88e9,3.8,Christian,Male,17
Veteran,FALSE,b674dd7ee0d24096d1c019615ce4d12b20fcbff12d79d3c5a9d2118eb7ccbb28,0.0,Blake,Male,17
Amateur,TRUE,23fe711e0e3b77f1da7aa221ab1192afe21648d47d2b4fa7a5a659ff443a0eb5,0.7,Flora,Female,21
Regular,TRUE,7dc01f10bf20671ecfccdac23812b1b415acd42c2147cb0af4d48fcce2420f3e,0.1,Kylie,Male,21
Amateur,TRUE,f58aad5996a435f16b0284a3b267f973f9af99e7a89bee0430055a44fa92f977,0.0,Adrian,Female,17


### Overall Summary Table Basing on Experience's Categories


In [4]:
summarized_player_data <- player_data |>
    group_by(experience) |>
    summarize(num_players = n(),
              num_subscribed = sum(subscribe, na.rm = TRUE),
              avg_played_hours = mean(played_hours, na.rm = TRUE), 
              max_played_hours = max(played_hours, na.rm = TRUE), 
              min_played_hours = min(played_hours, na.rm = TRUE), 
              avg_age = mean(Age, na.rm = TRUE), 
              max_age = max(Age, na.rm = TRUE), 
              min_age = min(Age, na.rm = TRUE))
summarized_player_data

experience,num_players,num_subscribed,avg_played_hours,max_played_hours,min_played_hours,avg_age,max_age,min_age
<chr>,<int>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
Amateur,63,45,6.0174603,150.0,0,21.36508,57,11
Beginner,35,27,1.2485714,23.7,0,21.65714,42,17
Pro,14,10,2.6000000,30.3,0,16.92308,25,9
Regular,36,29,18.2083333,223.1,0,22.02857,58,10
Veteran,48,33,0.6479167,12.5,0,20.95833,46,16


#### Potential Issues with Data
1. Data may contain missing values such as **NA** in `Age` and **"prefer not to say"** in `gender`.
2. The `avg_played_hours` may have no representative of the player if the **time span** for data recordings are too short. For example, if the data is collected during summer, children may have more freetime playing games, while adults are less likely.
3. Due to a **smaller sample**, **outliers** may increase the average by too much, although this may not be the case across most other players in the same category. `Regular` category may be an example.
4. **Bias** in `experience` when self-reporting from players.

**2. "sessions.csv" is dataset with 5 variables that describes player's states in the session**
|Variable Name |Type|Meaning|
|:------------:|:--:|:-----:|
|`hashedEmail`|chr|Regristration email|
|`start_time`|chr|Connect time (in datetime)|
|`end_time`|chr|Disconnect time (in datetime)|
|`original_start_time`|dbl|Connect time (in Unix timestamp)|
|`original_end_time`|dbl|Disconnect time (in Unix timestamp)|

The **Unix timestamp** make calculation of total time spend on the server convient to calculate, with the formula (`original_end_time` - `original_start_time`) milliseconds.

In [8]:
session_data <- read_csv("data/sessions.csv")
head(session_data)

Rows: 1535 Columns: 5
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (3): hashedEmail, start_time, end_time
dbl (2): original_start_time, original_end_time

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


[1] "2024-06-30 17:53:20 UTC"

hashedEmail,start_time,end_time,original_start_time,original_end_time
<chr>,<chr>,<chr>,<dbl>,<dbl>
bfce39c89d6549f2bb94d8064d3ce69dc3d7e72b38f431d8aa0c4bf95ccee6bf,30/06/2024 18:12,30/06/2024 18:24,1.71977e+12,1.71977e+12
36d9cbb4c6bc0c1a6911436d2da0d09ec625e43e6552f575d4acc9cf487c4686,17/06/2024 23:33,17/06/2024 23:46,1.71867e+12,1.71867e+12
f8f5477f5a2e53616ae37421b1c660b971192bd8ff77e3398304c7ae42581fdc,25/07/2024 17:34,25/07/2024 17:57,1.72193e+12,1.72193e+12
bfce39c89d6549f2bb94d8064d3ce69dc3d7e72b38f431d8aa0c4bf95ccee6bf,25/07/2024 03:22,25/07/2024 03:58,1.72188e+12,1.72188e+12
36d9cbb4c6bc0c1a6911436d2da0d09ec625e43e6552f575d4acc9cf487c4686,25/05/2024 16:01,25/05/2024 16:12,1.71665e+12,1.71665e+12
bfce39c89d6549f2bb94d8064d3ce69dc3d7e72b38f431d8aa0c4bf95ccee6bf,23/06/2024 15:08,23/06/2024 17:10,1.71916e+12,1.71916e+12
